In [ ]:
import pandas as pd

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display, HTML
import itertools
from functools import reduce
import scipy.stats
from copy import deepcopy
from pybedtools import BedTool
import pickle
from tqdm import tqdm
from multiprocessing import Pool
from pyfaidx import Fasta
from seq2atac.analysis.mutation_processing_pipeline_utils import annotateNearestGene, annotateScore
from seq2atac.stable import compute_gc_bed, one_hot_encode, plot_weights_given_ax, read_pickle, compute_signal, write_pickle
from seq2atac.stable.models.convolutional import get_bpnet_model
from seq2atac.analysis import get_promoterai_tss, get_cosmic_pancan_genes, sizesfile, fasta_file
from seq2atac.analysis.enrichment_utils import compute_ref_alt_scores, load_tracks
from seq2atac.analysis.enrichment_utils import create_pancancer_distribution_plots, create_pancancer_correlations, create_pancancer_valuecounts
from seq2atac.analysis.sample_controls import matching_logic
from seq2atac.analysis.mutation_utils import search_names_in_vierstra_group, compute_vierstra_groups, compute_trinuc, compute_motif_hits, intersect_motif_file, compute_gene_annotation, compute_closest_gene_annotation, ingene_indicator

myround = lambda x:np.format_float_scientific(x, precision=3)
all_tcga_cancers = ["BLCA","BRCA","COAD","GBM","KIRC","KIRP","LUAD","SKCM"]

### differential peaks - annotate closest gene

In [ ]:
df_45b_peaks = pd.read_csv("/illumina/scratch/deep_learning/lsundaram/singlecelldatasets/TCGA/SubClonal_Identification/GBM45_cloneB_peaks.csv")[["seqnames","start","end"]]
# df_45b_peaks = pd.read_csv("/illumina/scratch/deep_learning/lsundaram/singlecelldatasets/TCGA/SubClonal_Identification/GBM45_cloneB_peaks.csv")[["seqnames","start","end"]]
df_45b_peaks["summit"] = (df_45b_peaks["start"] + df_45b_peaks["end"])//2
df_45b_peaks["summit_1"] = df_45b_peaks["summit"] + 1
df_45b_peaks["peak_id"] = df_45b_peaks.index
df_45b_peaks


In [ ]:
peaksize=500
df_45b_peaks["start"] = df_45b_peaks["start"] + (500-peaksize)//2
df_45b_peaks["end"] = df_45b_peaks["start"] + peaksize

In [ ]:
peak_bed = BedTool.from_dataframe(df_45b_peaks[["seqnames","summit","summit_1","peak_id"]])
tss_df = get_promoterai_tss()
tss_df
tss_bed = BedTool.from_dataframe(tss_df)

In [ ]:
peak_annotated = peak_bed.sort().closest(tss_bed.sort(), d=True, t="first").to_dataframe(names=["seqnames","summit","summit_1","peak_id","chr","tss","tss_1","gene","distance_to_tss"])
peak_annotated

In [ ]:
df_45b_peaks_merged = df_45b_peaks.merge(peak_annotated, how="left", on=["seqnames","summit","summit_1","peak_id"])
assert len(df_45b_peaks_merged) == len(df_45b_peaks)
df_45b_peaks_merged

### Read all cleaned motifs

In [ ]:
gbm45b_allmotifs = pd.read_csv('/illumina/scratch/deep_learning/akumar22/TCGA/vierstra_peaks/vierstra_Archetype_GBM45_cloneBagg.csv.bed')
gbm45b_allmotifs

In [ ]:
gbm45b_motifs = pd.read_csv('/illumina/scratch/deep_learning/nravindra/results/reg_diffs/ismshap_gbm_subclone_vierstrav1_GBM45_cloneB.csv')
print(len(gbm45b_motifs))
gbm45b_motifs = gbm45b_motifs.merge(gbm45b_allmotifs)
gbm45b_motifs

In [ ]:
gbm45b_motifs = gbm45b_motifs[["seqnames","start","end","group_name","individual_match","es"]]
gbm45b_motifs

### Get all cleaned motif instances overlapping differential peaks

In [ ]:
motif_bed = BedTool.from_dataframe(gbm45b_motifs)
peaks_bed = BedTool.from_dataframe(df_45b_peaks_merged[["seqnames","start","end"]])

motif_bed_intersected = motif_bed.intersect(peaks_bed, wa=True, wb=True).to_dataframe(names=["motif_chr","motif_start","motif_end","group_name","individual_match","es","seqnames","start","end"]).merge(df_45b_peaks_merged)
motif_bed_intersected

In [ ]:
### Subset to those instances whose archetype contains a Chr6 TF

In [ ]:
all_tfs = pd.read_csv("DatabaseExtract_v_1.01.csv")
display(all_tfs)
tss_df = pd.read_csv("/illumina/scratch/deep_learning/nersaro/promoterAI/data/ref_data/gencodev39_cage_ratio_to_sum_refined_tss_positions_transcripts_protein_coding_inclZeros_withTranscriptID.tsv",sep="\t")
display(tss_df)

In [ ]:
tf_chromosome = "chr6"

In [ ]:
chr6tfs = sorted(list(tss_df[(tss_df["gene"].isin(all_tfs["HGNC symbol"])) & (tss_df["chrom"]==tf_chromosome)]["gene"].unique()))
print(len(chr6tfs))
chr6tfs

In [ ]:
all_individual_motif_names = list(motif_bed_intersected["individual_match"].unique())
all_individual_motif_names

In [ ]:
chr6_individual_names = []
for gene in chr6tfs:
    for ind_name in all_individual_motif_names:
        
        if gene.upper() in ind_name.upper():
            
            chr6_individual_names.append(ind_name)
            
len(chr6_individual_names)

In [ ]:
chr6_individual_names

In [ ]:
indiviual_name_to_archetype_dict = dict(zip(motif_bed_intersected["individual_match"],
                                            motif_bed_intersected["group_name"]))

In [ ]:
write_pickle(indiviual_name_to_archetype_dict,"indiviual_name_to_archetype_dict.pkl")

In [ ]:
chr6_tf_group_names = []

for ind_name in chr6_individual_names:
    
    chr6_tf_group_names.append(indiviual_name_to_archetype_dict[ind_name])

chr6_tf_group_names = list(set(chr6_tf_group_names))
print(chr6_tf_group_names)
sorted(chr6_tf_group_names)

In [ ]:
write_pickle(motif_bed_intersected,"motif_bed_intersected_presubsetting.pkl")

In [ ]:
motif_bed_intersected = read_pickle("motif_bed_intersected_presubsetting.pkl")
motif_bed_intersected

In [ ]:
motif_bed_intersected = motif_bed_intersected[(motif_bed_intersected["group_name"].isin(chr6_tf_group_names))]
motif_bed_intersected


In [ ]:
motif_bed_intersected = motif_bed_intersected.drop_duplicates(["peak_id"])
motif_bed_intersected

In [ ]:
motif_bed_intersected["tf_not"] = motif_bed_intersected["gene"].apply(lambda g : int(g in all_tfs["HGNC symbol"].tolist()))
motif_bed_intersected

In [ ]:
l1 = len(motif_bed_intersected[motif_bed_intersected["tf_not"]==1])
l2 = len(motif_bed_intersected[motif_bed_intersected["tf_not"]==0])
l3 = len(all_tfs)
total_numgenes = len(pd.read_csv("/illumina/scratch/deep_learning/public_data/refdata/hg38/genes/gencodev39/gencode.v39.CAGEcanonical.protein_coding.incl_chrX.tsv", sep="\t"))
l4 = total_numgenes-len(all_tfs)

In [ ]:
contingency = [[l1,l2],[l3,l4]]
contingency

In [ ]:
scipy.stats.fisher_exact(contingency)

In [ ]:
# l1 = len(motif_bed_intersected[(motif_bed_intersected["tf_not"]==1) & (motif_bed_intersected["distance_to_tss"] <= 250000)])
# l2 = len(motif_bed_intersected[(motif_bed_intersected["tf_not"]==0) & (motif_bed_intersected["distance_to_tss"] <= 250000)])
# l3 = len(motif_bed_intersected[(motif_bed_intersected["tf_not"]==1) & (motif_bed_intersected["distance_to_tss"] > 250000)])
# l4 = len(motif_bed_intersected[(motif_bed_intersected["tf_not"]==0) & (motif_bed_intersected["distance_to_tss"] > 250000)])


In [ ]:
# contingency = [[l1,l2],[l3,l4]]
# contingency

In [ ]:
# scipy.stats.fisher_exact(contingency)

In [ ]:
motif_bed_intersected = read_pickle("motif_bed_intersected_presubsetting.pkl")
motif_bed_intersected

In [ ]:
motif_bed_intersected = motif_bed_intersected[(motif_bed_intersected["individual_match"].isin(chr6_individual_names))]
motif_bed_intersected


In [ ]:
interest = motif_bed_intersected[(motif_bed_intersected["individual_match"].isin(["SOX4_HUMAN.H11MO.0.B", "SOX4_HMG_1"])) & 
                                  (motif_bed_intersected["gene"].isin(all_tfs["HGNC symbol"])) &
                                (motif_bed_intersected["distance_to_tss"] < 10000) ].sort_values("es",ascending=False)
interest

In [ ]:
interest = interest[interest["gene"]=="MYCL"]
interest

In [ ]:
### before this step, read the master motif_bed_intersected
motif_bed_intersected[(motif_bed_intersected["seqnames"]=="chr1") &
                      (motif_bed_intersected["peak_id"]==2068) & 
                      (motif_bed_intersected["motif_start"] >= 39898399-100) &
                      (motif_bed_intersected["motif_end"] <= 39898412+100)]

In [ ]:
from seq2atac.stable import one_hot_encode
from seq2atac.analysis.shap_utils import score_classification, compute_shap_score

In [ ]:
from seq2atac.analysis.shap_utils import plot_summit_centered, plot_mutation_centered, predict_classification_proba, plot_peak
import glob
from seq2atac.stable.models.convolutional import get_bpnet_model
model = get_bpnet_model(1364,8)
fasta_seq = Fasta(fasta_file)

In [ ]:
cancer_name = "GBM45_cloneB"
weights_files = glob.glob(f"../../../models_250_1364_minibatch_prejitter/gbm_subclone/{cancer_name}/fold_*/model.h5")
weights_files

In [ ]:
import matplotlib
plt.rcParams["figure.figsize"]=20,10
matplotlib.rcParams['pdf.fonttype']=42
!mkdir brca_vignette_figure/

In [ ]:
!mkdir sox4_near_mycl_vignette

In [ ]:
### centering on summit
chrom = interest.iloc[0]["seqnames"]
start = interest.iloc[0]["start"]
end = interest.iloc[0]["end"]

input_size = 1364
summit = (start+end)//2
start = summit - input_size//2
end = start + input_size
X_ref = one_hot_encode([fasta_seq[chrom][start:end]])

score1 = score_classification(X_ref,model,weights_files,compute_shap_score) * X_ref
prob_recalc = predict_classification_proba(X_ref, model, weights_files)[0]
print("score: ",prob_recalc)
np.save("sox4_near_mycl_vignette_summit_centered.npy",score1)

In [ ]:
row = interest.iloc[0]
for flank in [25,50,100,250]:
    
    # set up plot
    fig,axes = plt.subplots(figsize=(30*flank/100,3))
    mid_pt = 1364//2
    plt_start = mid_pt-flank
    plt_end = mid_pt+flank
    
    score1 = np.load("sox4_near_mycl_vignette_summit_centered.npy")
    plot_weights_given_ax(axes,score1[0][plt_start:plt_end], subticks_frequency=20)

    motif_start, motif_end = row["motif_start"], row["motif_end"]
    motif_len = motif_end - motif_start
    motif_relative_to_summit = motif_start - (summit - flank)
    axes.axvspan(motif_relative_to_summit, motif_relative_to_summit+motif_len, alpha=0.2, color='gray')
    
    ymax = score1[0][plt_start:plt_end].max()
    axes.set_ylim(-ymax,ymax)
    
    fig_title = f"{cancer_name} {chrom}:{start}-{end}"
        
    fig.suptitle(fig_title)
    fig.tight_layout()
    
    plt.savefig(f'sox4_near_mycl_vignette/summit_centered_{flank}.pdf',dpi=1200)
    
    

In [ ]:
### centering on motif
chrom = interest.iloc[0]["motif_chr"]
start = interest.iloc[0]["motif_start"]
end = interest.iloc[0]["motif_end"]

input_size = 1364
summit = (start+end)//2
start = summit - input_size//2
end = start + input_size
X_ref = one_hot_encode([fasta_seq[chrom][start:end]])

score2 = score_classification(X_ref,model,weights_files,compute_shap_score) * X_ref
prob_recalc = predict_classification_proba(X_ref, model, weights_files)[0]
print("score: ",prob_recalc)
np.save("sox4_near_mycl_vignette_motif_centered.npy",score2)

In [ ]:
row = interest.iloc[0]
for flank in [25,50,100,250]:
    
    # set up plot
    fig,axes = plt.subplots(figsize=(30*flank/100,3))
    mid_pt = 1364//2
    plt_start = mid_pt-flank
    plt_end = mid_pt+flank
    
    score1 = np.load("sox4_near_mycl_vignette_motif_centered.npy")
    plot_weights_given_ax(axes,score1[0][plt_start:plt_end], subticks_frequency=20)

    motif_start, motif_end = row["motif_start"], row["motif_end"]
    motif_len = motif_end - motif_start
    motif_relative_to_summit = motif_start - (summit - flank)
    axes.axvspan(motif_relative_to_summit, motif_relative_to_summit+motif_len, alpha=0.2, color='gray')
    
    ymax = score1[0][plt_start:plt_end].max()
    axes.set_ylim(-ymax,ymax)
    
    fig_title = f"{cancer_name} {chrom}:{start}-{end}"
        
    fig.suptitle(fig_title)
    fig.tight_layout()
    
    plt.savefig(f'sox4_near_mycl_vignette/motif_centered_{flank}.pdf',dpi=1200)
    
    